## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## The task

In [6]:
task = '''
        Write a concise but engaging blogpost about
        the benefits of AI. Make sure the blogpost is
       within 100 words.
       '''

## Create a writer agent

In [7]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [8]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [9]:
print(reply)

Title: Unleashing the Power of AI: A Game-Changer for Businesses

In today's fast-paced world, Artificial Intelligence (AI) is revolutionizing industries across the globe. From enhancing customer experiences to optimizing operations, the benefits of AI are undeniable. It boosts productivity, saves time, and cuts costs, ultimately driving profitability. With AI's ability to analyze vast amounts of data swiftly, businesses can make informed decisions and stay ahead of the curve. Additionally, AI streamlines processes, improves accuracy, and even opens up new revenue streams. Embracing AI isn't just a trend; it's a strategic move that propels businesses into a future of innovation and success.


## Adding Reflection

In [10]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [11]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
        the benefits of AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of AI: A Game-Changer for Businesses

In today's fast-paced world, Artificial Intelligence (AI) is revolutionizing industries across the globe. From enhancing customer experiences to optimizing operations, the benefits of AI are undeniable. It boosts productivity, saves time, and cuts costs, ultimately driving profitability. With AI's ability to analyze vast amounts of data swiftly, businesses can make informed decisions and stay ahead of the curve. Additionally, AI streamlines processes, improves accuracy, and even opens up new revenue streams. Embracing AI isn't just a trend; it's a strategic move that propels businesses into a future of innovation and success.

-----------------------------------

## Nested Chat

In [16]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [12]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [13]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [14]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [17]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [18]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [19]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
        the benefits of AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of AI: A Game-Changer for Businesses

In today's fast-paced world, Artificial Intelligence (AI) is revolutionizing industries across the globe. From enhancing customer experiences to optimizing operations, the benefits of AI are undeniable. It boosts productivity, saves time, and cuts costs, ultimately driving profitability. With AI's ability to analyze vast amounts of data swiftly, businesses can make informed decisions and stay ahead of the curve. Additionally, AI streamlines processes, improves accuracy, and even opens up new revenue streams. Embracing AI isn't just a trend; it's a strategic move that propels businesses into a future of innovation and success.

-----------------------------------


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregated Feedback:
1. SEO Reviewers (both) suggested incorporating specific keywords related to AI throughout the content for better search engine visibility. They also recommended utilizing header tags to organize the content and adding a meta description focusing on AI benefits for businesses.
2. The Ethics Reviewer did not identify any specific ethical issues but emphasized the importance of accurate representation of AI benefits and functionalities to prevent misleading claims. Transparency about potential limitations and ethical considerations when using AI in business was also advised.

Final Suggestions on the Writing:
1. Incorporate specific AI-related keywords as suggested by the SEO Reviewers to improve search engine visibility.
2. Use header tags (H2, H3) to structure the content, making it more reader-friendly and SEO-friendly.
3. Include a meta description highli

## Summary

In [20]:
print(res.summary)

Title: Unlocking Business Success: The Transformational Power of AI

In the digital era, Artificial Intelligence (AI) emerges as a pivotal tool, reshaping industries worldwide. AI optimizes operations, enhances customer experiences, and accelerates growth. Its capacity to swiftly analyze vast data sets empowers businesses to make strategic decisions, gain competitive edges, and boost profitability. Through streamlined processes and elevated accuracy, AI paves the way for revenue diversification. Embracing AI isn't just a strategy; it's a leap towards innovative success. Let's delve into the profound benefits AI offers, manifesting a future where businesses thrive amidst constant evolution.
